<a href="https://colab.research.google.com/github/Ravenha/Videogame_project/blob/master/Copy_of_Bethany_assignment_applied_modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Applied Modeling, Module 1

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Choose which observations you will use to train, validate, and test your model. And which observations, if any, to exclude.
- [ ] Determine whether your problem is regression or classification.
- [ ] Choose your evaluation metric.
- [ ] Begin with baselines: majority class baseline for classification, or mean baseline for regression, with your metric of choice.
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" information from the future?

## Reading

### ROC AUC
- [Machine Learning Meets Economics](http://blog.mldb.ai/blog/posts/2016/01/ml-meets-economics/)
- [ROC curves and Area Under the Curve explained](https://www.dataschool.io/roc-curves-and-auc-explained/)
- [The philosophical argument for using ROC curves](https://lukeoakdenrayner.wordpress.com/2018/01/07/the-philosophical-argument-for-using-roc-curves/)

### Imbalanced Classes
- [imbalance-learn](https://github.com/scikit-learn-contrib/imbalanced-learn)
- [Learning from Imbalanced Classes](https://www.svds.com/tbt-learning-imbalanced-classes/)

### Last lesson
- [Attacking discrimination with smarter machine learning](https://research.google.com/bigpicture/attacking-discrimination-in-ml/), by Google Research, with  interactive visualizations. _"A threshold classifier essentially makes a yes/no decision, putting things in one category or another. We look at how these classifiers work, ways they can potentially be unfair, and how you might turn an unfair classifier into a fairer one. As an illustrative example, we focus on loan granting scenarios where a bank may grant or deny a loan based on a single, automatically computed number such as a credit score."_
- [How Shopify Capital Uses Quantile Regression To Help Merchants Succeed](https://engineering.shopify.com/blogs/engineering/how-shopify-uses-machine-learning-to-help-our-merchants-grow-their-business)
- [Maximizing Scarce Maintenance Resources with Data: Applying predictive modeling, precision at k, and clustering to optimize impact](https://towardsdatascience.com/maximizing-scarce-maintenance-resources-with-data-8f3491133050), by Lambda DS3 student Michael Brady. His blog post extends the Tanzania Waterpumps scenario, far beyond what's in the lecture notebook.
- [Notebook about how to calculate expected value from a confusion matrix by treating it as a cost-benefit matrix](https://github.com/podopie/DAT18NYC/blob/master/classes/13-expected_value_cost_benefit_analysis.ipynb)
- [Simple guide to confusion matrix terminology](https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/) by Kevin Markham, with video
- [Visualizing Machine Learning Thresholds to Make Better Business Decisions](https://blog.insightdatascience.com/visualizing-machine-learning-thresholds-to-make-better-business-decisions-4ab07f823415)

In [5]:
# Import libraries
%matplotlib inline
import io
import pandas_profiling
import category_encoders as ce
import pandas as pd
import requests
import urllib.request
import time
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import sklearn; sklearn.__version__
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib import pyplot
from sklearn.decomposition import PCA
from collections import OrderedDict
from datetime import date
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from itertools import groupby
from math import sqrt
from numpy import linalg as LA
from mpl_toolkits import mplot3d
from numpy.linalg import matrix_rank
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
# from mlxtend.plotting import category_scatter
from sklearn.linear_model import LinearRegression
from ipywidgets import interact
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import classification_report

In [8]:
df1 = pd.read_csv('Vidgame.csv')
df1.head()

TypeError: __init__() got an unexpected keyword argument 'max_rows'

   Unnamed: 0  Rank                                  Name_x   Genre_x  \
0           0    31             Grand Theft Auto: Vice City    Action   
1           1    54                    Grand Theft Auto III    Action   
2           2   122                                 FIFA 13    Sports   
3           3   125  Grand Theft Auto: Liberty City Stories    Action   
4           4   129  Crash Bandicoot 2: Cortex Strikes Back  Platform   

  ESRB_Rating Platform_x                  Publisher_x     Developer_x  \
0           M        PS2               Rockstar Games  Rockstar North   
1           M        PS2               Rockstar Games      DMA Design   
2           E        PS3                    EA Sports       EA Canada   
3           M        PSP               Rockstar Games  Rockstar Leeds   
4           E         PS  Sony Computer Entertainment     Naughty Dog   

   Critic_Score_x  User_Score_x   ...    NA_Sales_y  EU_Sales  JP_Sales_y  \
0             9.6           NaN   ...        

In [9]:
# i will be predicting Global sales based on Platform, Genre, and Critic_Score
#this is a Regression Model -  regression prediction problems are usually quantities or sizes.For example, when provided with a dataset, and you are asked to predict their prices, that is a regression task 
#mean baseline for regression - 0.645525

In [10]:
df1.shape

(6819850, 32)

In [11]:
df1.describe()

TypeError: __init__() got an unexpected keyword argument 'max_rows'

         Unnamed: 0          Rank  Critic_Score_x  User_Score_x  \
count  6.819850e+06  6.819850e+06   914069.000000  30551.000000   
mean   3.409924e+06  1.476555e+04        6.717516      7.821944   
std    1.968721e+06  3.795738e+03        1.511179      1.508209   
min    0.000000e+00  3.100000e+01        1.000000      2.000000   
25%    1.704962e+06  1.260900e+04        5.900000      7.100000   
50%    3.409924e+06  1.572300e+04        7.000000      8.000000   
75%    5.114887e+06  1.772600e+04        7.900000      8.800000   
max    6.819849e+06  1.986200e+04       10.000000     10.000000   

       Total_Shipped  Global_Sales    NA_Sales_x     PAL_Sales    JP_Sales_x  \
count            0.0  6.819850e+06  3.711870e+06  3.535753e+06  2.677664e+06   
mean             NaN  8.299457e-02  7.264411e-02  3.381188e-02  5.046756e-02   
std              NaN  1.277889e-01  9.558789e-02  5.992161e-02  6.879039e-02   
min              NaN  1.000000e-02  0.000000e+00  0.000000e+00  0.000000e+00

In [12]:
df1.columns

Index(['Unnamed: 0', 'Rank', 'Name_x', 'Genre_x', 'ESRB_Rating', 'Platform_x',
       'Publisher_x', 'Developer_x', 'Critic_Score_x', 'User_Score_x',
       'Total_Shipped', 'Global_Sales', 'NA_Sales_x', 'PAL_Sales',
       'JP_Sales_x', 'Other_Sales_x', 'Year', 'Name_y', 'Platform_y',
       'Year_of_Release', 'Genre_y', 'Publisher_y', 'NA_Sales_y', 'EU_Sales',
       'JP_Sales_y', 'Other_Sales_y', 'Critic_Score_y', 'Critic_Count',
       'User_Score_y', 'User_Count', 'Developer_y', 'Rating'],
      dtype='object')

In [13]:
# Split into train & test sets first
# Seed for reproduceability
X_train, X_test = train_test_split(df1, train_size=0.8, test_size=0.2, 
                                  )

# Split X_train into train and val sets
# Seed for reproduceability
X_train, X_val = train_test_split(X_train, train_size=0.8, test_size=0.2, 
                                 )

# Drop target column from train, validation, & test sets and set y target vectors for each
# Our goal is to predict the Global_Sales based on the features provided
target = 'Global_Sales'
y_train = X_train[target]
y_val = X_val[target]
y_test = X_test[target]

X_train = X_train.drop(columns=target)
X_val = X_val.drop(columns=target)
X_test = X_test.drop(columns=target)

# REMEMBER: test set will not be used until the end!
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((4364704, 31),
 (4364704,),
 (1091176, 31),
 (1091176,),
 (1363970, 31),
 (1363970,))